In [ ]:
! pip install -r requirements.txt

In [2]:
import os
import json
import requests
import time

from openai import AzureOpenAI
from pathlib import Path
from typing import Optional
from IPython.display import clear_output

from dotenv import load_dotenv

load_dotenv(override=True)

True

In [3]:
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")
aoai_api_key = os.getenv("AZURE_OPENAI_KEY")
deployment_name = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

bing_search_subscription_key = os.getenv("BING_SEARCH_SUBSCRIPTION_KEY")
bing_search_url = f"{os.getenv('BING_SEARCH_URL')}v7.0/search"

In [4]:
def bing_search(query: str) -> list:
    """
    Perform a bing search against the given query

    @param query: Search query
    @return: List of search results

    """
    headers = {"Ocp-Apim-Subscription-Key": bing_search_subscription_key}
    params = {"q": query, "textDecorations": False}
    response = requests.get(bing_search_url, headers=headers, params=params)
    response.raise_for_status()
    search_results = response.json()

    output = []

    for result in search_results["webPages"]["value"]:
        output.append({"title": result["name"], "link": result["url"], "snippet": result["snippet"]})

    return json.dumps(output)


# bing_search("who is the ceo of Goldman Sachs")

In [5]:
name = "websearch-assistant"
instructions = """
You are an assistant designed to help people answer questions.
You have access to query the web using Bing Search.
You should call bing search whenever a question requires up to date information or could benefit from web data.
Always include the Citation when looking at bing search with URL.
Answer in nice Markdown format.
"""

messages = [
    {"role": "system", "content": instructions},
    {"role": "user", "content": "Who is the CEO of Goldman Sachs?"},
]

tools = [
    {
        "type": "function",
        "function": {
            "name": "search_bing",
            "description": "Searches bing to get up-to-date information from the web.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query",
                    }
                },
                "required": ["query"],
            },
        },
    }
]

# Note: the JSON response may not always be valid; be sure to handle errors
available_functions = {
    "search_bing": bing_search,
}

client = AzureOpenAI(api_key=aoai_api_key, api_version=api_version, azure_endpoint=azure_endpoint)

current_response = client.chat.completions.create(
    model=deployment_name,
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)
# print(current_response)
response_message = current_response.choices[0].message
tool_calls = response_message.tool_calls
messages.append(response_message)

is_calling_function = True if tool_calls else False
print(f"calling function: {is_calling_function}")

while is_calling_function:

    tool_call = tool_calls[0]

    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(**function_args)
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )
    current_response = client.chat.completions.create(
        model=deployment_name,
        messages=messages,
        tools=tools,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    response_message = current_response.choices[0].message
    # print(response_message)
    messages.append(response_message)
    tool_calls = response_message.tool_calls
    is_calling_function = True if tool_calls else False

print("messages:")
print(messages[-1].content)


calling function: True
messages:
The CEO of Goldman Sachs is David M. Solomon. He has been in the position since October 2018.

For more information, you can visit his [Wikipedia page](https://en.wikipedia.org/wiki/David_M._Solomon) or his [profile on the Goldman Sachs website](https://www.goldmansachs.com/our-firm/our-people-and-leadership/leadership/executive-officers/david-solomon).
